## **Import**

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 12.6 MB/s eta 0:00:00


In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    roc_auc_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
)

import optuna
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None)
pd.set_option('display.max_info_columns', 200)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


## **Data**

In [6]:
df = pd.read_pickle("/content/drive/MyDrive/Semester 3/DL/Shiv/final_df.pkl")

In [7]:
df = df[[
       'image',
       'ViewPosition',
       'Pneumonia',
       ]].copy()

## **Model using CNN**


In [ ]:

df["ViewPosition"] = df["ViewPosition"].fillna("Unknown")
df["view_position_id"] = df["ViewPosition"].astype("category").cat.codes
n_view_positions = df["view_position_id"].nunique()
print("Number of distinct view positions:", n_view_positions)

train_df, temp_df = train_test_split(
    df,
    test_size=0.3,
    stratify=df["Pneumonia"],
    random_state=42,
)

val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    stratify=temp_df["Pneumonia"],
    random_state=42,
)

print(f"Train size: {len(train_df)}, Val size: {len(val_df)}, Test size: {len(test_df)}")
print("Train label distribution:\n", train_df["Pneumonia"].value_counts(normalize=True))
print("Val label distribution:\n", val_df["Pneumonia"].value_counts(normalize=True))
print("Test label distribution:\n", test_df["Pneumonia"].value_counts(normalize=True))


class XrayImageDataset(Dataset):
    """
    Returns: image_tensor, label, view_position_id
    - image_tensor: FloatTensor [3, 320, 320] (already normalized)
    - label: FloatTensor scalar (0.0 or 1.0)
    - view_position_id: LongTensor scalar (0..n_view_positions-1)
    """

    def __init__(self, dataframe: pd.DataFrame, augment: bool = False):
        self.df = dataframe.reset_index(drop=True)
        self.augment = augment

    def __len__(self):
        return len(self.df)

    def _to_tensor_chw(self, img):

        if torch.is_tensor(img):
            x = img.detach().float()
        elif isinstance(img, np.ndarray):
            x = torch.from_numpy(img).float()
        else:
            raise TypeError(f"Unsupported image type: {type(img)}")

        while x.ndim > 3:
            x = x.squeeze(0)

        if x.ndim == 2:
            x = x.unsqueeze(0)
        elif x.ndim == 3:
            if x.shape[0] not in (1, 3) and x.shape[-1] in (1, 3):
                x = x.permute(2, 0, 1)
        else:
            raise ValueError(f"Unexpected image shape: {x.shape}")

        C, H, W = x.shape

        if C == 1:
            x = x.repeat(3, 1, 1)
        elif C > 3:
            x = x[:3]

        if (H, W) != (320, 320):
            x = F.interpolate(
                x.unsqueeze(0), size=(320, 320), mode="bilinear", align_corners=False
            ).squeeze(0)

        return x

    def _augment_tensor(self, x):

        if not self.augment:
            return x

        if torch.rand(1).item() < 0.5:
            x = torch.flip(x, dims=[2])

        k = torch.randint(0, 4, (1,)).item()
        if k > 0:
            x = torch.rot90(x, k, dims=[1, 2])

        return x

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_data = row["image"]
        label = row["Pneumonia"]
        view_id = row["view_position_id"]

        x = self._to_tensor_chw(img_data)
        x = self._augment_tensor(x)

        y = torch.tensor(label, dtype=torch.float32)
        view_id = torch.tensor(view_id, dtype=torch.long)

        return x, y, view_id


# Create datasets
train_dataset = XrayImageDataset(train_df, augment=True)
val_dataset   = XrayImageDataset(val_df,   augment=False)
test_dataset  = XrayImageDataset(test_df,  augment=False)

# Default loaders
default_batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=default_batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=default_batch_size, shuffle=False)
test_loader  = DataLoader(test_dataset,  batch_size=default_batch_size, shuffle=False)


class CustomCNN(nn.Module):
    def __init__(
        self,
        in_channels: int = 3,
        num_blocks: int = 3,
        base_filters: int = 32,
        dropout_conv: float = 0.0,
        dropout_fc: float = 0.0,
        fc_hidden_dim: int = 128,
        n_view_positions: int = 3,
        view_emb_dim: int = 8,
    ):
        super().__init__()

        #  Convolutional blocks
        conv_layers = []
        curr_in = in_channels
        for b in range(num_blocks):
            out_ch = base_filters * (2**b)
            conv_layers.append(nn.Conv2d(curr_in, out_ch, kernel_size=3, padding=1))
            conv_layers.append(nn.BatchNorm2d(out_ch))
            conv_layers.append(nn.ReLU(inplace=True))
            if dropout_conv > 0.0:
                conv_layers.append(nn.Dropout2d(dropout_conv))
            conv_layers.append(nn.MaxPool2d(2, 2))
            curr_in = out_ch

        self.conv_blocks = nn.Sequential(*conv_layers)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))

        final_conv_ch = base_filters * (2 ** (num_blocks - 1))

        #   view_position embedding
        self.view_emb = nn.Embedding(num_embeddings=n_view_positions,
                                     embedding_dim=view_emb_dim)

        #   Fully connected head
        fc_in_dim = final_conv_ch + view_emb_dim

        if fc_hidden_dim is not None and fc_hidden_dim > 0:
            self.fc_layers = nn.Sequential(
                nn.Linear(fc_in_dim, fc_hidden_dim),
                nn.ReLU(inplace=True),
                nn.Dropout(dropout_fc) if dropout_fc > 0 else nn.Identity(),
                nn.Linear(fc_hidden_dim, 1),
            )
        else:
            self.fc_layers = nn.Linear(fc_in_dim, 1)

    def forward(self, x, view_ids):
        x = self.conv_blocks(x)
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)

        v = self.view_emb(view_ids)
        f = torch.cat([x, v], dim=1)
        logits = self.fc_layers(f)
        return logits


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


def train_model(
    model: nn.Module,
    train_loader: DataLoader,
    val_loader: DataLoader,
    device: torch.device,
    optimizer: torch.optim.Optimizer,
    criterion: nn.Module,
    num_epochs: int = 20,
    early_stopping_patience: int = 5,
):
    model.to(device)
    best_val_auc = 0.0
    best_state = None
    patience_counter = 0

    for epoch in range(1, num_epochs + 1):
        #   TRAIN
        model.train()
        train_loss_sum = 0.0

        for X, y, view_ids in train_loader:
            X = X.to(device)
            y = y.to(device)
            view_ids = view_ids.to(device)

            optimizer.zero_grad()
            logits = model(X, view_ids)
            loss = criterion(logits.view(-1), y)
            loss.backward()
            optimizer.step()

            train_loss_sum += loss.item() * X.size(0)

        avg_train_loss = train_loss_sum / len(train_loader.dataset)

        #   VALIDATION
        model.eval()
        val_loss_sum = 0.0
        all_logits = []
        all_targets = []

        with torch.no_grad():
            for X, y, view_ids in val_loader:
                X = X.to(device)
                y = y.to(device)
                view_ids = view_ids.to(device)

                logits = model(X, view_ids)
                loss = criterion(logits.view(-1), y)
                val_loss_sum += loss.item() * X.size(0)

                all_logits.append(logits.view(-1).cpu())
                all_targets.append(y.cpu())

        avg_val_loss = val_loss_sum / len(val_loader.dataset)
        all_logits = torch.cat(all_logits)
        all_targets = torch.cat(all_targets)

        val_probs = torch.sigmoid(all_logits)
        val_pred = (val_probs >= 0.5).int()

        val_acc = accuracy_score(all_targets.numpy(), val_pred.numpy())
        try:
            val_auc = roc_auc_score(all_targets.numpy(), val_probs.numpy())
        except ValueError:
            val_auc = 0.0

        print(
            f"Epoch {epoch}: "
            f"Train Loss {avg_train_loss:.4f} | "
            f"Val Loss {avg_val_loss:.4f} | "
            f"Val Acc {val_acc:.4f} | "
            f"Val ROC-AUC {val_auc:.4f}"
        )

        #   Early stopping
        if val_auc > best_val_auc:
            best_val_auc = val_auc
            best_state = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= early_stopping_patience:
            print("Early stopping triggered.")
            break

    if best_state is not None:
        model.load_state_dict(best_state)

    return model, best_val_auc


def objective(trial):
    # Hyperparameter search space
    lr = trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    num_blocks = trial.suggest_int("num_blocks", 2, 4)
    base_filters = trial.suggest_categorical("base_filters", [16, 32, 64])
    dropout_conv = trial.suggest_categorical("dropout_conv", [0.0, 0.2, 0.5])
    dropout_fc = trial.suggest_categorical("dropout_fc", [0.0, 0.3, 0.5])
    fc_hidden_dim = trial.suggest_categorical("fc_hidden_dim", [0, 64, 128, 256])
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)

    model = CustomCNN(
        in_channels=3,
        num_blocks=num_blocks,
        base_filters=base_filters,
        dropout_conv=dropout_conv,
        dropout_fc=dropout_fc,
        fc_hidden_dim=fc_hidden_dim,
        n_view_positions=n_view_positions,
        view_emb_dim=8,
    )

    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=lr,
        weight_decay=weight_decay,
    )
    criterion = nn.BCEWithLogitsLoss()

    # Trial-specific loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False)

    model, best_val_auc = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        device=device,
        optimizer=optimizer,
        criterion=criterion,
        num_epochs=15,
        early_stopping_patience=3,
    )

    return best_val_auc


# Create and run study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

print("Best hyperparameters:", study.best_params)
print("Best validation ROC-AUC:", study.best_value)


best_params = study.best_params
print("Training final model with best params:", best_params)

val_dataset  = XrayImageDataset(val_df,  augment=False)
test_dataset = XrayImageDataset(test_df, augment=False)

val_loader  = DataLoader(val_dataset,  batch_size=best_params["batch_size"], shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=best_params["batch_size"], shuffle=False)

train_val_df = pd.concat([train_df, val_df]).reset_index(drop=True)
train_val_dataset = XrayImageDataset(train_val_df, augment=True)
train_val_loader  = DataLoader(
    train_val_dataset,
    batch_size=best_params["batch_size"],
    shuffle=True,
)

final_model = CustomCNN(
    in_channels=3,
    num_blocks=best_params["num_blocks"],
    base_filters=best_params["base_filters"],
    dropout_conv=best_params["dropout_conv"],
    dropout_fc=best_params["dropout_fc"],
    fc_hidden_dim=best_params["fc_hidden_dim"],
    n_view_positions=n_view_positions,
    view_emb_dim=8,
).to(device)

optimizer = torch.optim.Adam(
    final_model.parameters(),
    lr=best_params["learning_rate"],
    weight_decay=best_params.get("weight_decay", 0.0),
)
criterion = nn.BCEWithLogitsLoss()

final_model, _ = train_model(
    model=final_model,
    train_loader=train_val_loader,
    val_loader=val_loader,
    device=device,
    optimizer=optimizer,
    criterion=criterion,
    num_epochs=10,
    early_stopping_patience=3,
)

#   Test evaluation
final_model.eval()
all_logits = []
all_labels = []
test_loss_sum = 0.0

with torch.no_grad():
    for X, y, view_ids in test_loader:
        X = X.to(device)
        y = y.to(device)
        view_ids = view_ids.to(device)

        logits = final_model(X, view_ids)
        loss = criterion(logits.view(-1), y)
        test_loss_sum += loss.item() * X.size(0)

        all_logits.append(logits.view(-1).cpu())
        all_labels.append(y.cpu())

all_logits = torch.cat(all_logits)
all_labels = torch.cat(all_labels)

test_loss = test_loss_sum / len(test_loader.dataset)
test_probs = torch.sigmoid(all_logits)
test_pred  = (test_probs >= 0.5).int()

test_acc  = accuracy_score(all_labels.numpy(), test_pred.numpy())
test_prec = precision_score(all_labels.numpy(), test_pred.numpy())
test_rec  = recall_score(all_labels.numpy(), test_pred.numpy())
test_f1   = f1_score(all_labels.numpy(), test_pred.numpy())
try:
    test_auc = roc_auc_score(all_labels.numpy(), test_probs.numpy())
except ValueError:
    test_auc = 0.0

cm = confusion_matrix(all_labels.numpy(), test_pred.numpy())
report = classification_report(all_labels.numpy(), test_pred.numpy(),
                               target_names=["Normal", "Pneumonia"])

print(f"\n=== Test Results ===")
print(f"Test Loss:      {test_loss:.4f}")
print(f"Test Accuracy:  {test_acc:.4f}")
print(f"Test ROC-AUC:   {test_auc:.4f}")
print(f"Test Precision: {test_prec:.4f}")
print(f"Test Recall:    {test_rec:.4f}")
print(f"Test F1-score:  {test_f1:.4f}")
print("\nConfusion Matrix:\n", cm)
print("\nClassification Report:\n", report)


[I 2025-12-07 19:42:38,470] A new study created in memory with name: no-name-5ee8b621-9b95-4c91-ba64-02a1362f8672


Number of distinct view positions: 2
Train size: 1734, Val size: 372, Test size: 372
Train label distribution:
 Pneumonia
0    0.531142
1    0.468858
Name: proportion, dtype: float64
Val label distribution:
 Pneumonia
0    0.52957
1    0.47043
Name: proportion, dtype: float64
Test label distribution:
 Pneumonia
0    0.532258
1    0.467742
Name: proportion, dtype: float64
Using device: cuda
Epoch 1: Train Loss 0.7015 | Val Loss 0.6978 | Val Acc 0.4758 | Val ROC-AUC 0.4456
Epoch 2: Train Loss 0.6998 | Val Loss 0.6954 | Val Acc 0.4624 | Val ROC-AUC 0.4848
Epoch 3: Train Loss 0.6957 | Val Loss 0.6958 | Val Acc 0.4624 | Val ROC-AUC 0.4745
Epoch 4: Train Loss 0.6961 | Val Loss 0.6956 | Val Acc 0.5296 | Val ROC-AUC 0.4414


[I 2025-12-07 19:43:26,707] Trial 0 finished with value: 0.4848150833937636 and parameters: {'learning_rate': 0.0002569335732560429, 'batch_size': 8, 'num_blocks': 4, 'base_filters': 16, 'dropout_conv': 0.5, 'dropout_fc': 0.5, 'fc_hidden_dim': 128, 'weight_decay': 1.9191223837893584e-06}. Best is trial 0 with value: 0.4848150833937636.


Epoch 5: Train Loss 0.6941 | Val Loss 0.6966 | Val Acc 0.5269 | Val ROC-AUC 0.4654
Early stopping triggered.
Epoch 1: Train Loss 0.6921 | Val Loss 0.6897 | Val Acc 0.5296 | Val ROC-AUC 0.5603
Epoch 2: Train Loss 0.6908 | Val Loss 0.6899 | Val Acc 0.5296 | Val ROC-AUC 0.5214
Epoch 3: Train Loss 0.6896 | Val Loss 0.6903 | Val Acc 0.5296 | Val ROC-AUC 0.5050


[I 2025-12-07 19:44:03,997] Trial 1 finished with value: 0.5602610587382162 and parameters: {'learning_rate': 1.59491200702303e-05, 'batch_size': 16, 'num_blocks': 3, 'base_filters': 32, 'dropout_conv': 0.0, 'dropout_fc': 0.0, 'fc_hidden_dim': 64, 'weight_decay': 8.171167539691874e-06}. Best is trial 1 with value: 0.5602610587382162.


...

Epoch 14: Train Loss 0.6816 | Val Loss 0.6903 | Val Acc 0.5430 | Val ROC-AUC 0.5475
Early stopping triggered.
Epoch 1: Train Loss 0.6922 | Val Loss 0.6960 | Val Acc 0.4651 | Val ROC-AUC 0.4263
Epoch 2: Train Loss 0.6909 | Val Loss 0.6949 | Val Acc 0.4677 | Val ROC-AUC 0.4336
Epoch 3: Train Loss 0.6902 | Val Loss 0.6949 | Val Acc 0.4624 | Val ROC-AUC 0.4381
Epoch 4: Train Loss 0.6896 | Val Loss 0.6946 | Val Acc 0.5081 | Val ROC-AUC 0.4432
Epoch 5: Train Loss 0.6893 | Val Loss 0.6945 | Val Acc 0.5054 | Val ROC-AUC 0.4441
Epoch 6: Train Loss 0.6891 | Val Loss 0.6946 | Val Acc 0.5188 | Val ROC-AUC 0.4470
Epoch 7: Train Loss 0.6887 | Val Loss 0.6949 | Val Acc 0.5188 | Val ROC-AUC 0.4479
Epoch 8: Train Loss 0.6885 | Val Loss 0.6952 | Val Acc 0.5134 | Val ROC-AUC 0.4495
Epoch 9: Train Loss 0.6883 | Val Loss 0.6952 | Val Acc 0.5242 | Val ROC-AUC 0.4504
Epoch 10: Train Loss 0.6879 | Val Loss 0.6955 | Val Acc 0.5081 | Val ROC-AUC 0.4505
Epoch 11: Train Loss 0.6880 | Val Loss 0.6959 | Val Acc 0.5

[I 2025-12-07 20:10:37,241] Trial 18 finished with value: 0.45606961566352433 and parameters: {'learning_rate': 1.7741767506694658e-05, 'batch_size': 32, 'num_blocks': 2, 'base_filters': 32, 'dropout_conv': 0.0, 'dropout_fc': 0.0, 'fc_hidden_dim': 64, 'weight_decay': 4.3701412934605116e-05}. Best is trial 17 with value: 0.5616823785351703.


Epoch 15: Train Loss 0.6872 | Val Loss 0.6961 | Val Acc 0.5188 | Val ROC-AUC 0.4557
Epoch 1: Train Loss 0.6917 | Val Loss 0.6901 | Val Acc 0.5242 | Val ROC-AUC 0.5191
Epoch 2: Train Loss 0.6891 | Val Loss 0.6905 | Val Acc 0.5161 | Val ROC-AUC 0.5072
Epoch 3: Train Loss 0.6874 | Val Loss 0.6907 | Val Acc 0.5430 | Val ROC-AUC 0.5075


[I 2025-12-07 20:11:14,266] Trial 19 finished with value: 0.5191007976794779 and parameters: {'learning_rate': 3.544986586260537e-05, 'batch_size': 16, 'num_blocks': 3, 'base_filters': 32, 'dropout_conv': 0.0, 'dropout_fc': 0.0, 'fc_hidden_dim': 128, 'weight_decay': 4.5634739959686986e-05}. Best is trial 17 with value: 0.5616823785351703.


Epoch 4: Train Loss 0.6871 | Val Loss 0.6903 | Val Acc 0.5430 | Val ROC-AUC 0.5163
Early stopping triggered.
Best hyperparameters: {'learning_rate': 6.905064059372074e-05, 'batch_size': 16, 'num_blocks': 3, 'base_filters': 32, 'dropout_conv': 0.0, 'dropout_fc': 0.0, 'fc_hidden_dim': 64, 'weight_decay': 4.47879326895593e-05}
Best validation ROC-AUC: 0.5616823785351703
Training final model with best params: {'learning_rate': 6.905064059372074e-05, 'batch_size': 16, 'num_blocks': 3, 'base_filters': 32, 'dropout_conv': 0.0, 'dropout_fc': 0.0, 'fc_hidden_dim': 64, 'weight_decay': 4.47879326895593e-05}
Epoch 1: Train Loss 0.6914 | Val Loss 0.6903 | Val Acc 0.5215 | Val ROC-AUC 0.5258
Epoch 2: Train Loss 0.6884 | Val Loss 0.6914 | Val Acc 0.5296 | Val ROC-AUC 0.5199
Epoch 3: Train Loss 0.6881 | Val Loss 0.6888 | Val Acc 0.5538 | Val ROC-AUC 0.5463
Epoch 4: Train Loss 0.6874 | Val Loss 0.6871 | Val Acc 0.5457 | Val ROC-AUC 0.5627
Epoch 5: Train Loss 0.6872 | Val Loss 0.6874 | Val Acc 0.5618 | 

## **Model using Resnet**

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    roc_auc_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
)

import optuna
import torchvision.models as models


df["ViewPosition"] = df["ViewPosition"].fillna("Unknown")
df["view_position_id"] = df["ViewPosition"].astype("category").cat.codes
n_view_positions = df["view_position_id"].nunique()
print("Number of distinct view positions:", n_view_positions)

train_df, temp_df = train_test_split(
    df,
    test_size=0.3,
    stratify=df["Pneumonia"],
    random_state=42,
)

val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    stratify=temp_df["Pneumonia"],
    random_state=42,
)

print(f"Train size: {len(train_df)}, Val size: {len(val_df)}, Test size: {len(test_df)}")
print("Train label distribution:\n", train_df["Pneumonia"].value_counts(normalize=True))
print("Val label distribution:\n", val_df["Pneumonia"].value_counts(normalize=True))
print("Test label distribution:\n", test_df["Pneumonia"].value_counts(normalize=True))


class XrayImageDataset(Dataset):
    """
    Returns: image_tensor, label, view_position_id
    - image_tensor: FloatTensor [3, 320, 320] (already normalized)
    - label: FloatTensor scalar (0.0 or 1.0)
    - view_position_id: LongTensor scalar (0..n_view_positions-1)
    """

    def __init__(self, dataframe: pd.DataFrame, augment: bool = False):
        self.df = dataframe.reset_index(drop=True)
        self.augment = augment

    def __len__(self):
        return len(self.df)

    def _to_tensor_chw(self, img):
        """
        Convert df['image'] entry into FloatTensor [3, 320, 320].
        Handles numpy arrays and torch tensors, with shapes like:
        - [3, 320, 320]
        - [1, 320, 320]
        - [320, 320]
        - [H, W, C]
        etc.
        Assumes intensities are ALREADY normalized; we do NOT re-normalize.
        """
        # 1) to tensor
        if torch.is_tensor(img):
            x = img.detach().float()
        elif isinstance(img, np.ndarray):
            x = torch.from_numpy(img).float()
        else:
            raise TypeError(f"Unsupported image type: {type(img)}")

        while x.ndim > 3:
            x = x.squeeze(0)

        if x.ndim == 2:
            x = x.unsqueeze(0)
        elif x.ndim == 3:
            if x.shape[0] not in (1, 3) and x.shape[-1] in (1, 3):
                x = x.permute(2, 0, 1)
        else:
            raise ValueError(f"Unexpected image shape: {x.shape}")

        C, H, W = x.shape

        if C == 1:
            x = x.repeat(3, 1, 1)
        elif C > 3:
            x = x[:3]

        if (H, W) != (320, 320):
            x = F.interpolate(
                x.unsqueeze(0), size=(320, 320), mode="bilinear", align_corners=False
            ).squeeze(0)

        return x

    def _augment_tensor(self, x):

        if not self.augment:
            return x

        if torch.rand(1).item() < 0.5:
            x = torch.flip(x, dims=[2])

        k = torch.randint(0, 4, (1,)).item()
        if k > 0:
            x = torch.rot90(x, k, dims=[1, 2])

        return x

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_data = row["image"]
        label = row["Pneumonia"]
        view_id = row["view_position_id"]

        x = self._to_tensor_chw(img_data)
        x = self._augment_tensor(x)

        y = torch.tensor(label, dtype=torch.float32)
        view_id = torch.tensor(view_id, dtype=torch.long)

        return x, y, view_id


# Create datasets
train_dataset = XrayImageDataset(train_df, augment=True)
val_dataset   = XrayImageDataset(val_df,   augment=False)
test_dataset  = XrayImageDataset(test_df,  augment=False)

# Default loaders
default_batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=default_batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=default_batch_size, shuffle=False)
test_loader  = DataLoader(test_dataset,  batch_size=default_batch_size, shuffle=False)

class ResNet18Custom(nn.Module):
    def __init__(
        self,
        n_view_positions: int,
        view_emb_dim: int = 8,
        fc_hidden_dim: int = 128,
        dropout_fc: float = 0.0,
    ):
        super().__init__()

        self.resnet = models.resnet18(weights=None)
        self.resnet.fc = nn.Identity()

        # view_position embedding
        self.view_emb = nn.Embedding(
            num_embeddings=n_view_positions,
            embedding_dim=view_emb_dim,
        )

        # Fully connected head
        feat_dim = 512
        fc_in_dim = feat_dim + view_emb_dim

        if fc_hidden_dim is not None and fc_hidden_dim > 0:
            self.fc_layers = nn.Sequential(
                nn.Linear(fc_in_dim, fc_hidden_dim),
                nn.ReLU(inplace=True),
                nn.Dropout(dropout_fc) if dropout_fc > 0 else nn.Identity(),
                nn.Linear(fc_hidden_dim, 1),
            )
        else:
            self.fc_layers = nn.Linear(fc_in_dim, 1)

    def forward(self, x, view_ids):
        img_feat = self.resnet(x)
        img_feat = img_feat.view(img_feat.size(0), -1)

        v = self.view_emb(view_ids)

        f = torch.cat([img_feat, v], dim=1)
        logits = self.fc_layers(f)
        return logits


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


def train_model(
    model: nn.Module,
    train_loader: DataLoader,
    val_loader: DataLoader,
    device: torch.device,
    optimizer: torch.optim.Optimizer,
    criterion: nn.Module,
    num_epochs: int = 20,
    early_stopping_patience: int = 5,
):
    model.to(device)
    best_val_auc = 0.0
    best_state = None
    patience_counter = 0

    for epoch in range(1, num_epochs + 1):
        #  TRAIN
        model.train()
        train_loss_sum = 0.0

        for X, y, view_ids in train_loader:
            X = X.to(device)
            y = y.to(device)
            view_ids = view_ids.to(device)

            optimizer.zero_grad()
            logits = model(X, view_ids)
            loss = criterion(logits.view(-1), y)
            loss.backward()
            optimizer.step()

            train_loss_sum += loss.item() * X.size(0)

        avg_train_loss = train_loss_sum / len(train_loader.dataset)

        #  VALIDATION
        model.eval()
        val_loss_sum = 0.0
        all_logits = []
        all_targets = []

        with torch.no_grad():
            for X, y, view_ids in val_loader:
                X = X.to(device)
                y = y.to(device)
                view_ids = view_ids.to(device)

                logits = model(X, view_ids)
                loss = criterion(logits.view(-1), y)
                val_loss_sum += loss.item() * X.size(0)

                all_logits.append(logits.view(-1).cpu())
                all_targets.append(y.cpu())

        avg_val_loss = val_loss_sum / len(val_loader.dataset)
        all_logits = torch.cat(all_logits)
        all_targets = torch.cat(all_targets)

        val_probs = torch.sigmoid(all_logits)
        val_pred = (val_probs >= 0.5).int()

        val_acc = accuracy_score(all_targets.numpy(), val_pred.numpy())
        try:
            val_auc = roc_auc_score(all_targets.numpy(), val_probs.numpy())
        except ValueError:
            val_auc = 0.0

        print(
            f"Epoch {epoch}: "
            f"Train Loss {avg_train_loss:.4f} | "
            f"Val Loss {avg_val_loss:.4f} | "
            f"Val Acc {val_acc:.4f} | "
            f"Val ROC-AUC {val_auc:.4f}"
        )

        #  Early stopping
        if val_auc > best_val_auc:
            best_val_auc = val_auc
            best_state = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= early_stopping_patience:
            print("Early stopping triggered.")
            break

    if best_state is not None:
        model.load_state_dict(best_state)

    return model, best_val_auc


def objective(trial):
    # Hyperparameter search space
    lr = trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    dropout_fc = trial.suggest_categorical("dropout_fc", [0.0, 0.3, 0.5])
    fc_hidden_dim = trial.suggest_categorical("fc_hidden_dim", [0, 64, 128, 256])
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)

    model = ResNet18Custom(
        n_view_positions=n_view_positions,
        view_emb_dim=8,
        fc_hidden_dim=fc_hidden_dim,
        dropout_fc=dropout_fc,
    )

    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=lr,
        weight_decay=weight_decay,
    )
    criterion = nn.BCEWithLogitsLoss()

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False)

    model, best_val_auc = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        device=device,
        optimizer=optimizer,
        criterion=criterion,
        num_epochs=15,
        early_stopping_patience=3,
    )

    return best_val_auc


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

print("Best hyperparameters:", study.best_params)
print("Best validation ROC-AUC:", study.best_value)


best_params = study.best_params
print("Training final ResNet-18 model with best params:", best_params)

val_dataset  = XrayImageDataset(val_df,  augment=False)
test_dataset = XrayImageDataset(test_df, augment=False)

val_loader  = DataLoader(val_dataset,  batch_size=best_params["batch_size"], shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=best_params["batch_size"], shuffle=False)

train_val_df = pd.concat([train_df, val_df]).reset_index(drop=True)
train_val_dataset = XrayImageDataset(train_val_df, augment=True)
train_val_loader  = DataLoader(
    train_val_dataset,
    batch_size=best_params["batch_size"],
    shuffle=True,
)

# Build final model
final_model = ResNet18Custom(
    n_view_positions=n_view_positions,
    view_emb_dim=8,
    fc_hidden_dim=best_params["fc_hidden_dim"],
    dropout_fc=best_params["dropout_fc"],
).to(device)

optimizer = torch.optim.Adam(
    final_model.parameters(),
    lr=best_params["learning_rate"],
    weight_decay=best_params.get("weight_decay", 0.0),
)
criterion = nn.BCEWithLogitsLoss()

# Train final model
final_model, _ = train_model(
    model=final_model,
    train_loader=train_val_loader,
    val_loader=val_loader,
    device=device,
    optimizer=optimizer,
    criterion=criterion,
    num_epochs=10,
    early_stopping_patience=3,
)

# Test evaluation
final_model.eval()
all_logits = []
all_labels = []
test_loss_sum = 0.0

with torch.no_grad():
    for X, y, view_ids in test_loader:
        X = X.to(device)
        y = y.to(device)
        view_ids = view_ids.to(device)

        logits = final_model(X, view_ids)
        loss = criterion(logits.view(-1), y)
        test_loss_sum += loss.item() * X.size(0)

        all_logits.append(logits.view(-1).cpu())
        all_labels.append(y.cpu())

all_logits = torch.cat(all_logits)
all_labels = torch.cat(all_labels)

test_loss = test_loss_sum / len(test_loader.dataset)
test_probs = torch.sigmoid(all_logits)
test_pred  = (test_probs >= 0.5).int()

test_acc  = accuracy_score(all_labels.numpy(), test_pred.numpy())
test_prec = precision_score(all_labels.numpy(), test_pred.numpy())
test_rec  = recall_score(all_labels.numpy(), test_pred.numpy())
test_f1   = f1_score(all_labels.numpy(), test_pred.numpy())
try:
    test_auc = roc_auc_score(all_labels.numpy(), test_probs.numpy())
except ValueError:
    test_auc = 0.0

cm = confusion_matrix(all_labels.numpy(), test_pred.numpy())
report = classification_report(all_labels.numpy(), test_pred.numpy(),
                               target_names=["Normal", "Pneumonia"])

print(f"\n=== Test Results ===")
print(f"Test Loss:      {test_loss:.4f}")
print(f"Test Accuracy:  {test_acc:.4f}")
print(f"Test ROC-AUC:   {test_auc:.4f}")
print(f"Test Precision: {test_prec:.4f}")
print(f"Test Recall:    {test_rec:.4f}")
print(f"Test F1-score:  {test_f1:.4f}")
print("\nConfusion Matrix:\n", cm)
print("\nClassification Report:\n", report)


[I 2025-12-08 21:15:32,135] A new study created in memory with name: no-name-94a3b13a-270b-4ed2-ba2e-16a5182d1d02


Number of distinct view positions: 2
Train size: 1734, Val size: 372, Test size: 372
Train label distribution:
 Pneumonia
0    0.531142
1    0.468858
Name: proportion, dtype: float64
Val label distribution:
 Pneumonia
0    0.52957
1    0.47043
Name: proportion, dtype: float64
Test label distribution:
 Pneumonia
0    0.532258
1    0.467742
Name: proportion, dtype: float64
Using device: cuda
Epoch 1: Train Loss 0.6919 | Val Loss 0.6855 | Val Acc 0.5591 | Val ROC-AUC 0.5780
Epoch 2: Train Loss 0.6812 | Val Loss 0.6984 | Val Acc 0.5457 | Val ROC-AUC 0.5791
Epoch 3: Train Loss 0.6727 | Val Loss 0.6749 | Val Acc 0.5484 | Val ROC-AUC 0.6265
Epoch 4: Train Loss 0.6639 | Val Loss 0.6907 | Val Acc 0.5538 | Val ROC-AUC 0.6209
Epoch 5: Train Loss 0.6585 | Val Loss 0.6959 | Val Acc 0.5618 | Val ROC-AUC 0.6138


[I 2025-12-08 21:16:53,619] Trial 0 finished with value: 0.6265409717186367 and parameters: {'learning_rate': 1.1099458182818129e-05, 'batch_size': 32, 'dropout_fc': 0.0, 'fc_hidden_dim': 0, 'weight_decay': 9.600574798366024e-06}. Best is trial 0 with value: 0.6265409717186367.


Epoch 6: Train Loss 0.6544 | Val Loss 0.7078 | Val Acc 0.5699 | Val ROC-AUC 0.6080
Early stopping triggered.
Epoch 1: Train Loss 0.7017 | Val Loss 0.6895 | Val Acc 0.5565 | Val ROC-AUC 0.5374
Epoch 2: Train Loss 0.6913 | Val Loss 0.6918 | Val Acc 0.5027 | Val ROC-AUC 0.5279
Epoch 3: Train Loss 0.6876 | Val Loss 0.7136 | Val Acc 0.4704 | Val ROC-AUC 0.5956
Epoch 4: Train Loss 0.6893 | Val Loss 0.6900 | Val Acc 0.5296 | Val ROC-AUC 0.5577
Epoch 5: Train Loss 0.6892 | Val Loss 0.6944 | Val Acc 0.5081 | Val ROC-AUC 0.5095


[I 2025-12-08 21:18:10,007] Trial 1 finished with value: 0.5955910079767949 and parameters: {'learning_rate': 0.0008222883331103353, 'batch_size': 16, 'dropout_fc': 0.3, 'fc_hidden_dim': 64, 'weight_decay': 9.393866191827285e-05}. Best is trial 0 with value: 0.6265409717186367.


...

Epoch 13: Train Loss 0.6676 | Val Loss 0.7187 | Val Acc 0.5081 | Val ROC-AUC 0.6082
Early stopping triggered.
Epoch 1: Train Loss 0.6969 | Val Loss 0.6897 | Val Acc 0.5054 | Val ROC-AUC 0.5201
Epoch 2: Train Loss 0.6830 | Val Loss 0.6846 | Val Acc 0.5565 | Val ROC-AUC 0.5802
Epoch 3: Train Loss 0.6823 | Val Loss 0.6788 | Val Acc 0.5645 | Val ROC-AUC 0.5917
Epoch 4: Train Loss 0.6758 | Val Loss 0.6719 | Val Acc 0.5914 | Val ROC-AUC 0.6067
Epoch 5: Train Loss 0.6717 | Val Loss 0.7045 | Val Acc 0.5645 | Val ROC-AUC 0.5935
Epoch 6: Train Loss 0.6706 | Val Loss 0.6771 | Val Acc 0.5349 | Val ROC-AUC 0.6013
Epoch 7: Train Loss 0.6682 | Val Loss 0.7031 | Val Acc 0.5511 | Val ROC-AUC 0.6157
Epoch 8: Train Loss 0.6717 | Val Loss 0.6732 | Val Acc 0.5699 | Val ROC-AUC 0.6098
Epoch 9: Train Loss 0.6731 | Val Loss 0.6739 | Val Acc 0.5565 | Val ROC-AUC 0.6006
Epoch 10: Train Loss 0.6705 | Val Loss 0.6617 | Val Acc 0.5753 | Val ROC-AUC 0.6436
Epoch 11: Train Loss 0.6592 | Val Loss 0.6793 | Val Acc 0.5

[I 2025-12-08 21:52:33,839] Trial 18 finished with value: 0.6585641769398115 and parameters: {'learning_rate': 8.584901857613793e-05, 'batch_size': 8, 'dropout_fc': 0.5, 'fc_hidden_dim': 128, 'weight_decay': 3.756748704136713e-05}. Best is trial 15 with value: 0.6591443074691806.


Epoch 15: Train Loss 0.6455 | Val Loss 0.6691 | Val Acc 0.5699 | Val ROC-AUC 0.6351
Epoch 1: Train Loss 0.6974 | Val Loss 0.6894 | Val Acc 0.5699 | Val ROC-AUC 0.5334
Epoch 2: Train Loss 0.6945 | Val Loss 0.6942 | Val Acc 0.4812 | Val ROC-AUC 0.5269
Epoch 3: Train Loss 0.6905 | Val Loss 0.6922 | Val Acc 0.5296 | Val ROC-AUC 0.5441
Epoch 4: Train Loss 0.6909 | Val Loss 0.6912 | Val Acc 0.5269 | Val ROC-AUC 0.5245
Epoch 5: Train Loss 0.6892 | Val Loss 0.6898 | Val Acc 0.4731 | Val ROC-AUC 0.5225
Epoch 6: Train Loss 0.6892 | Val Loss 0.6899 | Val Acc 0.5161 | Val ROC-AUC 0.5581
Epoch 7: Train Loss 0.6865 | Val Loss 0.6884 | Val Acc 0.5296 | Val ROC-AUC 0.5670
Epoch 8: Train Loss 0.6844 | Val Loss 0.6853 | Val Acc 0.5565 | Val ROC-AUC 0.5702
Epoch 9: Train Loss 0.6855 | Val Loss 0.6843 | Val Acc 0.5672 | Val ROC-AUC 0.5679
Epoch 10: Train Loss 0.6816 | Val Loss 0.6969 | Val Acc 0.5188 | Val ROC-AUC 0.5880
Epoch 11: Train Loss 0.6887 | Val Loss 0.6911 | Val Acc 0.5296 | Val ROC-AUC 0.5172
E

[I 2025-12-08 21:55:45,946] Trial 19 finished with value: 0.5880203045685279 and parameters: {'learning_rate': 0.00042270642458186935, 'batch_size': 8, 'dropout_fc': 0.5, 'fc_hidden_dim': 128, 'weight_decay': 5.389741352273169e-05}. Best is trial 15 with value: 0.6591443074691806.


Epoch 13: Train Loss 0.6854 | Val Loss 0.6912 | Val Acc 0.5484 | Val ROC-AUC 0.5335
Early stopping triggered.
Best hyperparameters: {'learning_rate': 0.000203735992061931, 'batch_size': 32, 'dropout_fc': 0.5, 'fc_hidden_dim': 128, 'weight_decay': 5.513772513711078e-05}
Best validation ROC-AUC: 0.6591443074691806
Training final ResNet-18 model with best params: {'learning_rate': 0.000203735992061931, 'batch_size': 32, 'dropout_fc': 0.5, 'fc_hidden_dim': 128, 'weight_decay': 5.513772513711078e-05}
Epoch 1: Train Loss 0.6966 | Val Loss 0.6779 | Val Acc 0.5618 | Val ROC-AUC 0.6128
Epoch 2: Train Loss 0.6783 | Val Loss 0.6790 | Val Acc 0.5968 | Val ROC-AUC 0.5968
Epoch 3: Train Loss 0.6789 | Val Loss 0.6680 | Val Acc 0.5753 | Val ROC-AUC 0.6422
Epoch 4: Train Loss 0.6764 | Val Loss 0.6992 | Val Acc 0.4892 | Val ROC-AUC 0.5890
Epoch 5: Train Loss 0.6762 | Val Loss 0.6720 | Val Acc 0.6102 | Val ROC-AUC 0.6305
Epoch 6: Train Loss 0.6722 | Val Loss 0.6586 | Val Acc 0.5941 | Val ROC-AUC 0.6403
E